### Retriever & Prompt Template 연결

In [ ]:
"""

< Prompt Template >
LLM에게 사전에 역할/명령을 부여하는 도구

< Retriever >
백터 저장소에서 문서를 검색하는 도구

"""

In [1]:
## Stage 1.
## LLM + PromptTemplate 사용 예시:

# Libraries
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough

# LLM Load
local_model = "wizardlm2"
llm = ChatOllama(model=local_model)

# Template 구축
template = """You are an AI assistant.
              Answer the question based on your knowledge.
              If you do not know the answer, just say you don't know.
              Do not share false information.
              Question: {question}
              """

# PromptTemplate + LLM = Chain
llm_template = PromptTemplate.from_template(template)
chain = (
    {"question" : RunnablePassthrough()}
    | llm_template
    | llm
    | StrOutputParser()
)

In [2]:
# 질문
ans = chain.invoke("현재 미국 대통령은 누구야?")
print(ans)

현재 미국의 대통령은 Joe Biden입니다. 그는 2021년 1월 20일에 입임하여 46번째 미국 대통령으로, 2020년 선거를 통해 승당받았습니다. Joe Biden은 바라크 오바마의 후계자로서 연임을 시도했으며, 이전에 미국 參회장(Vice President)으로 역할했던 경력이 있습니다.


In [24]:
## Stage 2.
## LLM + PromptTemplate + Retriever (Multi_Query) 사용 예시: 

# Libraries
from langchain.document_loaders import PyPDFLoader

# pdf read
local_path = "./data/케이뱅크.pdf"
loader = PyPDFLoader(local_path)
data = loader.load_and_split()
print(len(data))
data[:5]

20


[Document(page_content='은행  (NEUTRAL ) \n케이뱅크  – 2Q 분기 흑자 전환 성공  \n \n• 케이뱅크는  2Q21 28.1% q-q의 견조한  자산 성장률을  기록한  가운데 , 39억원의  흑자\n전환에  성공. 작년 분기별  평균 263억원의  적자 시현 감안할 때, 이익 개선이  빠르\n게 이뤄진  것을 방증. \n• 1) 대출 고성장으로  이자이익이  396.4 % y-y, 71.3% q-q 증가와 함께 2) 수수료이익\n이 85억원의  순이익을  기록한  것이 실적 개선을  견읶. \n• 지난 7월 완료된  1.25조원의 유상증자는  케이뱅크  실적의  본격적읶  개선을  위한 기\n반이 될 전망. 향후 성장의  지속성을  위한 관건은  고객 기반 활용도  제고로 판단.  \nWHAT’S THE STORY  \n케이뱅크  2Q21 실적 - 적자폭  크게 감소: 케이뱅크는  2Q21 28.1% q -q의 견조한  자산 성장\n률을 기록한  가욲데 , 39억원의  흑자젂홖에  성공. 참고로 , 동사가  작년 연갂 1,054 억원, 붂기\n별로는  평균 263억원의  적자를  시현한  점을 감앆할 때, 2Q 실적은  동사 이익 개선이  빠르\n게 이뤄짂  것을 방증. 이자이익이  증가와  수수료이익  증가가  실적 개선을  견읶. \n1. 이자이익  증가 – 대출 고성장  영향: 케이뱅크의  이자이익은  71.3% q -q 증가한  448억원\n을 기록. 2Q 이자이익  증가 폭은 186억원으로  지난 붂기 개선 폭 (97억원)에 비해 개선 속\n도가 빨라지고  있으며, 이자이익의  증가가  붂기 실적 흑자 젂홖에  주요 요읶으로  작용한  것\n으로 판단. \n2. 수수료  큰 폭 흑자 시현 – 업비트  관련 수수료  이익 증가에  기읶: 2Q 중 수수료이익  부\n문 흑자 시현. 젂붂기  1억원 순손실을  기록했으나 , 2Q에는 85억원의  순이익을  기록. 가입자  \n수 증가에  따른 비용 지출 증가에도  불구하고

In [25]:
# LLM load
from langchain.chat_models import ChatOllama
llm = ChatOllama(model="wizardlm2")

In [28]:
# Libraries
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings


# 한국어 - 임베딩 모델 사용
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {"normalize_embeddings":True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs
)

# Create Child Documents - text splitter - 자식 문서 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(data)


# VectorStore 구축
vectorstore = Chroma.from_documents(
            documents=splits,
            embedding=ko_embedding
            )


# MultiQuery-Retriever에 사용되는 QUERY_PROMPT 구축
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}
    """,
)

# Document Retriever : 
# 탐색기 [llm & prompt 기능이 추가된 Retriever 구축]
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm = llm,
    prompt=QUERY_PROMPT
)

In [29]:
## RAG Prompt Setting - Final prompt template for LLM
final_template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

# Chain 연결
prompt = PromptTemplate.from_template(final_template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
# 질문 문의
answer_from_llm = chain.invoke("5가지 bullet points로 요약해줘")
answer_from_llm

'케이뱅크의 보고서에서 추출한 주요 정보를 바탕으로, 다음과 같은 5가지 bullet point로 요약할 수 있습니다:\n\n1. 케이뱅크는 2021년 기준으로 총자산 12,049.2십억원을 보유하고 있으며, 주거용주택담보와 소매 익스포저(개익)에서 가장 큰 수익을 창출했습니다.\n2. 표준방법에 따른 적용 자산의 위험가중치별 익스포저 성과를 분석한 결과, 100% 차감 후의 소매 익스포저(개익)가 가장 높은 수익을 보였습니다.\n3. 판관비 추이 분석에서 케이뱅크는 2019년부터 2021년까지의 판관비 증가 트렌드를 파악하고, 시장 상황에 따라 판관비용률이 개선될 것으로 예상합니다.\n4. 케이뱅크는 2021년 기준 P/B(배)가 0.6, P/E(배)가 6.1, PER(배)가 5.1, PBR(배)가 0.4로 설정한 목표 지표를 바탕으로 시장 분석을 수행했습니다.\n5. 미래의 시장 성장과 판관비용률 개선에 대한 예측을 포함하여, 20A년부터 2022년까지의 PER(배)와 PBR(배) 예상치를 제시했습니다.\n\n이러한 요약은 보고서의 주요 내용을 기반으로 하며, 각 항목에 대한 더 자세한 정보는 원본 문서를 참조해야 합니다.'